In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
#this package is used to load matlab files to python notebook
from scipy.io import loadmat 

In [7]:
bs = 64
doc(untar_data)

In [9]:
path = untar_data(URLs.CARS); path

PosixPath('/home/paperspace/.fastai/data/stanford-cars')

In [10]:
path.ls()

[PosixPath('/home/paperspace/.fastai/data/stanford-cars/cars_test'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/labels.csv'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/cars_annos.mat')]

In [11]:
cars_anno = path/'cars_annos.mat'
cars_train_img = path/'car_ims'
cars_test_img = path/'cars_test'

In [13]:
fnames = get_image_files(cars_train_img)
fnames[:5]

[PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims/07200.jpg'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims/02631.jpg'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims/00346.jpg'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims/02549.jpg'),
 PosixPath('/home/paperspace/.fastai/data/stanford-cars/car_ims/07348.jpg')]

### Create Labels from class names

In [14]:
MAT = loadmat(cars_anno)
class_names = MAT['class_names']

In [15]:
labels = [c for c in class_names[0]]
labels = pd.DataFrame(labels, columns=['labels'])
labels.head()

,labels
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012


### Flatten annotations 

In [17]:
frame = [[i.flat[0] for i in line] for line in MAT['annotations'][0]]
columns = [ 'fname','bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class','test']
df_train = pd.DataFrame(frame, columns=columns)
df_train.head()

,fname,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test
0,car_ims/000001.jpg,112,7,853,717,1,0
1,car_ims/000002.jpg,48,24,441,202,1,0
2,car_ims/000003.jpg,7,4,277,180,1,0
3,car_ims/000004.jpg,33,50,197,150,1,0
4,car_ims/000005.jpg,5,8,83,58,1,0


In [11]:
print(MAT['annotations'][0][0].flat[0][0])

['car_ims/000001.jpg']


### Merge labels

In [12]:
df_train = df_train.merge(labels, left_on='class', right_index=True)
df_train = df_train.sort_index()
df_train.tail()

,fname,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test,labels
16100,car_ims/016101.jpg,37,48,365,266,195,1,smart fortwo Convertible 2012
16101,car_ims/016102.jpg,157,592,1300,1087,195,1,smart fortwo Convertible 2012
16102,car_ims/016103.jpg,62,1,587,340,195,1,smart fortwo Convertible 2012
16103,car_ims/016104.jpg,104,402,1220,817,195,1,smart fortwo Convertible 2012
16104,car_ims/016105.jpg,16,14,287,166,195,1,smart fortwo Convertible 2012


In [18]:
df_train.to_csv(path/'labels.csv', encoding='utf-8', index=False)

In [19]:
doc(ImageDataBunch)

In [23]:
from PIL import Image
from pathlib import Path
from matplotlib.patches import Rectangle
def get_assets(df, i):
    is_train = df is df_train
    folder = cars_train_img
    image = Image.open(path/'car_ims/16101.jpg')
    title = df['labels'][i] if is_train else 'Unclassified'

    xy = df['bbox_x1'][i], df['bbox_y1'][i]
    width = df['bbox_x2'][i] - df['bbox_x1'][i]
    height = df['bbox_y2'][i] - df['bbox_y1'][i]
    rect = Rectangle(xy, width, height, fill=False, color='r', linewidth=2)
    
    return (image, title, rect)

In [24]:

def display_image(df, i):
    image, title, rect = get_assets(df, i)
    print(title)

    plt.imshow(image)
    plt.axis('off')
    plt.title(title)
    plt.gca().add_patch(rect)

In [25]:
display_image(df_train, 0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/paperspace/.fastai/data/stanford-cars/car_ims/16101.jpg'